In [3]:
# default_exp algo.seq.embeding.glove

%reload_ext autoreload
%autoreload 2

# glove简介
GloVe: Global Vectors for Word Representation

GloVe是一种用于__获取单词向量__表示的无监督学习算法。 对来自语料库的汇总全局单词-单词共现统计信息进行训练，并且所得表示形式展示了单词向量空间的有趣线性子结构。




官网https://nlp.stanford.edu/projects/glove/

https://github.com/stanfordnlp/GloVe

# 原理
https://blog.csdn.net/u014665013/article/details/79642083

https://www.cnblogs.com/jfdwd/p/11086914.html

http://www.foldl.me/2014/glove-python/

* 首先基于语料库构建词的共现矩阵，
* 然后基于共现矩阵和GloVe模型学习词向量。

** 开始 -> 统计共现矩阵 -> 训练词向量 -> 结束**
## 构建共现矩阵X
所谓的共现，共同出现，其实就是看一个词有没有在另一个词的附近出现，所谓的附近，其实就是一个移动窗口的概念，定义窗口的半径（从中心词到边缘的距离）后，看看方圆多少范围内出现词的个数，就是共现，现在看看例子。

假设语料库就只有下面一行：

    i love you but you love him i am sad

设半径为2，于是移动窗口的滑动就有下面的形式：

以窗口5为例，此处就可以认为，love分别和but, you, him, i共同出现了一次，通过这种方式去计数，就能知道任意两个词之间的共现关系（一般是可逆的），构成共现矩阵X，一般地，X是一个对称矩阵。

X_ij的意义：在整个语料库中，单词i和单词j共同出现在一个窗口中的次数。
## 使用GloVe模型训练词向量
GloVe的训练目标是学习单词向量，使其点积等于单词共现概率的对数。  

由于比率的对数等于对数之差(log(a/b)=log(a)-log(b)，因此该目标将共现概率的比率（对数）与词向量空间中的向量差相关联。 

因为这些比率可以编码某种形式的含义，所以此信息也被编码为矢量差。 

因此，生成的词向量在词类比任务（例如在word2vec程序包中检查的类词）上的性能非常好。

公式表示为: vi和vj是词汇i和j的词向量，bi和bj是常数项，
$$v_i^Tv_j+b_j+b_i=log(X_{ij})$$

### glove模型的损失函数：
$$J=\sum_{i,j}^{N}f(X_{ij})(v_i^Tv_j+b_j+b_i-log(X_{ij}))^2$$

f是特定的权重函数, 我们选择一个f来帮助防止常见单词对（即具有较大Xij值的单词对）过于偏离我们的目标，N是词汇表大小。
$$f(X_{ij})=\begin{cases}(\frac{X_{ij}}{x_{max}})^\alpha, if X_{ij}<x_{max}\cr 1, othervise\end{cases}$$
当我们遇到极其常见的单词对（其中Xij> xmax）时，此函数将切断其正常输出并仅返回1。对于所有其他单词对，我们返回范围（0,1）内的一些权重，其中权重的分布范围由α决定。

# python实现
http://www.foldl.me/2014/glove-python/

python2实现: https://github.com/hans/glove.py

python3实现: https://github.com/maierhofert/glove.py

现在获取代码！ 我将跳过无聊的部分，这些部分完成诸如模型保存和参数解析之类的工作，而专注于代码中最复杂的三个函数：
* `build_cooccur`   
accepts a corpus and yields a list of co-occurrence blobs (the Xij values). It calculates co-occurrences by moving a sliding n-gram window over each sentence in the corpus.
* `train_glove`,   
which prepares the parameters of the model and manages training at a high level, and
* `run_iter`,   
which runs a single parameter update step.

首先，我们的build_cooccur函数接受一个词汇表（将单词映射为整数单词ID），一个语料库（一个简单的句子迭代器）和一些可选参数：上下文窗口大小和最小计数（用于删除稀有单词共现对 ）。 首先，我们将建立一个稀疏矩阵来收集同现Xij和一些简单的辅助数据。

接下来，train_glove在给定完全共现数据的情况下初始化模型参数。 我们期望与第一个参数相同的vocab对象。 第二个参数cooccurrences是在build_cooccur中生成的同现迭代器，它产生形式为（main_word_id，context_word_id，x_ij）的同现元组，其中x_ij是如上所述的Xij同现值。

    def train_glove(vocab, cooccurrences, vector_size=100,
                    iterations=25, **kwargs):
                    
接下来，我们准备主要的模型参数：词向量矩阵W和偏差标量的集合。 请注意，我们的单词矩阵的行数是语料库中单词数的两倍。 我们将在稍后描述run_iter函数时找出原因。                    
   
我们将使用自适应梯度下降（AdaGrad）[7]进行训练，因此我们还需要初始化W的辅助矩阵和跟踪梯度历史的偏差矢量。 请注意，所有这些都初始化为1的块。 从每个等于1的梯度历史开始，我们在AdaGrad的第一步培训将简单地为每个示例使用全局学习率。 （请参阅脚注77，以从AdaGrad定义中得出结论。）
                    
接下来，我们通过迭代调用run_iter函数开始训练。                    
                    
运行iter接受这些预取的数据，并开始对其进行重新排序，并为迭代建立一个全局成本global_cost                    
                    
现在，对于每个同现数据元组，我们按照上述理论部分中的描述计算加权成本。 每个元组具有以下元素：

    v_main：共现中主要单词的单词向量

    v_context：共现中上下文词的词向量

    b_main：主词的偏义标量

    b_context：上下文词的偏差标量

    gradsq_W_main：一个向量，用于存储主词向量的平方梯度历史记录（用于AdaGrad更新）

    gradsq_W_context：上下文词向量的向量梯度历史

    gradsq_b_main：主词偏向的标量梯度历史

    gradsq_b_context：上下文词偏差的标量梯度历史

    cooccurrence：Xij

我们保留中间的“内部”成本（不平方或加权），用于在下一节中计算梯度。

计算成本后，我们现在需要计算梯度。 从我们的原始成本函数J中，我们得出有关相关参数w⃗i，w⃗j，bi和bj的梯度。 （请注意，由于f（Xij）不依赖于这些参数中的任何一个，因此推导非常简单。）下面，我们使用运算符⊙来表示逐元素矢量乘法。

$$\begin{align*}J &= \sum_{i=1}^V \sum_{j=1}^V \; f\left(X_{ij}\right) \left( \vec{w}_i^T \vec{w}_j + b_i + b_j - \log X_{ij} \right)^2 \\ \nabla_{\vec{w}_i} J &= \sum_{j=1}^V f\left(X_{ij}\right) \vec{w}_j \odot \left( \vec{w}_i^T \vec{w}_j + b_i + b_j - \log X_{ij}\right) \\ \frac{\partial J}{\partial b_i} &= \sum_{j=1}^V f\left(X_{ij}\right) \left(\vec w_i^T \vec w_j + b_i + b_j - \log X_{ij}\right) \end{align*}$$


最后，我们使用AdaGrad7更新权重，并将计算出的梯度添加到梯度历史变量中。



## 官网glove的使用

The demo.sh script downloads a small corpus, consisting of the first 100M characters of Wikipedia. It collects unigram counts, constructs and shuffles cooccurrence data, and trains a simple version of the GloVe model. 

It also runs a word analogy evaluation script in python to verify word vector quality. More details about training on your own corpus can be found by reading demo.sh or the src/README.md

初始化参数

    CORPUS=text8
    VOCAB_FILE=vocab.txt
    COOCCURRENCE_FILE=cooccurrence.bin
    COOCCURRENCE_SHUF_FILE=cooccurrence.shuf.bin
    BUILDDIR=build
    SAVE_FILE=vectors
    VERBOSE=2
    MEMORY=4.0
    VOCAB_MIN_COUNT=5
    VECTOR_SIZE=50
    MAX_ITER=15
    WINDOW_SIZE=15
    
运行会新生成五个文件

    cooccurrence.bin 
    cooccurrence.shuf.bin
    vectors.bin
    vectors.txt  # 词向量
    vocab.txt  # 词汇表


In [10]:
!ls /Users/luoyonggui/Documents/temp/GloVe/ 

LICENSE               cooccurrence.shuf.bin text8
Makefile              demo.sh               text8.zip
README.md             eval                  vectors.bin
build                 randomization.test.sh vectors.txt
cooccurrence.bin      src                   vocab.txt


In [1]:
!cd /Users/luoyonggui/Documents/temp/GloVe/ && make

mkdir -p build
gcc -c src/vocab_count.c -o build/vocab_count.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
clang: warning: -lm: 'linker' input unused [-Wunused-command-line-argument]
gcc -c src/cooccur.c -o build/cooccur.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
clang: warning: -lm: 'linker' input unused [-Wunused-command-line-argument]
gcc -c src/shuffle.c -o build/shuffle.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
clang: warning: -lm: 'linker' input unused [-Wunused-command-line-argument]
gcc -c src/glove.c -o build/glove.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
clang: warning: -lm: 'linker' input unused [-Wunused-command-line-argument]
gcc -c src/common.c -o build/common.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
clang: warning: -lm: 'linker' input unused [-Wunused-command-line-argument]
gcc build/vocab_count.o build/common.o -o build/

In [6]:
!cd /Users/luoyonggui/Documents/temp/GloVe/ && ./demo.sh

mkdir -p build

$ build/vocab_count -min-count 5 -verbose 2 < text8 > vocab.txt
BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.1200000 tokens.1300000 tokens.1400000 tokens.1500000 tokens.1600000 tokens.1700000 tokens.1800000 tokens.1900000 tokens.2000000 tokens.2100000 tokens.2200000 tokens.2300000 tokens.2400000 tokens.2500000 tokens.2600000 tokens.2700000 tokens.2800000 tokens.2900000 tokens.3000000 tokens.3100000 tokens.3200000 tokens.3300000 tokens.3400000 tokens.3500000 tokens.3600000 tokens.3700000 tokens.3800000 tokens.3900000 tokens.4000000 tokens.4100000 tokens.4200000 tokens.4300000 tokens.4400000 tokens.4500000 tokens.4600000 tokens.4700000 tokens.4800000 tokens.4900000 tokens.5000000 tokens.5100000 tokens.5200000 tokens.5300000 tokens.5400000 tokens.5500000 tokens.5600000 tokens.5700000 tokens.5800000 tokens.5900000 tokens.60000

## install
注: 最新版本是1.0.2，但是安装报错

In [3]:
!pip install glove -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip freeze | grep glove

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 51kB 1.3MB/s eta 0:00:011
  ERROR: Complete output from command /Users/luoyonggui/anaconda3/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/private/var/folders/7j/kgtjln3x2dj2g2v57d5vncyw0000gp/T/pip-install-wc01u86_/glove/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/7j/kgtjln3x2dj2g2v57d5vncyw0000gp/T/pip-wheel-yejbsbdo --python-tag cp37:
  ERROR: running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.9-x86_64-3.7
  creating build/lib.macosx-10.9-x86_64-3.7/glove
  copying glove/__init__.py -> build/lib.macosx-10.9-x86_64-3.7/glove
  copying glove/glove.py -> build/lib.macosx-10.9-x86_64-3.7/glove
  running egg_info
  writing glove.egg-info/PKG-INFO
  writing dep

In [29]:
!pip uninstall glove -y

Uninstalling glove-1.0.0:
  Successfully uninstalled glove-1.0.0


In [31]:
!conda install glove_python 


CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/main/osx-64/repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.

If your current network has https://www.anaconda.com blocked, please file
a support request with your network engineering team.

ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='repo.anaconda.com', port=443): Max retries exceeded with url: /pkgs/main/osx-64/repodata.json (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x11754a898>, 'Connection to repo.anaconda.com timed out. (connect timeout=9.15)'))"))




In [32]:
!sudo ln -s /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX10.14.sdk


Password:


关键报错语句:

    Compiling with an SDK that doesn't seem to exist: /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX10.14.sdk
    
推断是因为mac没有安装Xcode，然后我的系统是mac 10.13.6. 对应的可以安装xcode 10.1 /xcode 9.4 我选择安装的是xcode10.1版本的. https://blog.csdn.net/weixin_30580943/article/details/99755640 https://www.jianshu.com/p/0e18f2db5acc

In [1]:
!pip install glove_python -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/3e/79/7e7e548dd9dcb741935d031117f4bed133276c2a047aadad42f1552d1771/glove_python-0.1.0.tar.gz
  ERROR: Complete output from command /Users/luoyonggui/anaconda3/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/private/var/folders/7j/kgtjln3x2dj2g2v57d5vncyw0000gp/T/pip-install-8jz7eiu1/glove-python/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/7j/kgtjln3x2dj2g2v57d5vncyw0000gp/T/pip-wheel-yf51v2bw --python-tag cp37:
  ERROR: running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.9-x86_64-3.7
  creating build/lib.macosx-10.9-x86_64-3.7/glove
  copying glove/__init__.py -> build/lib.macosx-10.9-x86_64-3.7/glove
  copying glove/glove.py -> build/lib.mac

## GloVe的Python实现

在pypi里面看到了很多GloVe的包，但是很多都有坑，我直接说一个我自己已经走通的包mittens。

https://github.com/roamanalytics/mittens

下载方式还是比较简单的， pip install mittens基本没什么问题，想要去看看源码的话，在这里：

一般而言GloVe按照计算共现矩阵和GloVe训练两大模块，而mittens里面其实只提供了后者，需要自己构造共现矩阵

In [2]:
# !conda install glove_python # xxx
# !pip install mittens
!pip freeze | grep mittens

mittens==0.2


In [3]:
from mittens import GloVe
# 初始化模型
vecLength=100           # 矩阵长度
max_iter=100000         # 最大迭代次数
display_progress=1000   # 每次展示

In [4]:
glove_model = GloVe(n=vecLength, max_iter=max_iter, display_progress=display_progress)


In [6]:
from mylib.algo.rs.matrix import load_co_occurance_matrix

In [7]:
save_dir = '../data/kdd_debiasing/data_gen/matrix_test/'
train_data_matrix, id2item, item2id = load_co_occurance_matrix(save_dir=save_dir, 
                                                      penalty1=True, penalty2=True,
                                                      penalty3=0.9)

In [ ]:
# 模型训练与结果输出
embeddings = glove_model.fit(train_data_matrix)

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/scipy/sparse/compressed.py:287: SparseEfficiencyWarning: Comparing a sparse matrix with a scalar greater than zero using <= is inefficient, try using > instead.
  warn(bad_scalar_msg, SparseEfficiencyWarning)


## gensim加载glove训练的词向量
https://radimrehurek.com/gensim/scripts/glove2word2vec.html

glove词向量的格式如下：

    word1 0.123 0.134 0.532 0.152
    word2 0.934 0.412 0.532 0.159
    word3 0.334 0.241 0.324 0.18
    ...
    word9 0.334 0.241 0.324 0.188

word2vec词向量的格式：

    9 4   # 这一行包含向量的数量及其维度
    word1 0.123 0.134 0.532 0.152
    word2 0.934 0.412 0.532 0.159
    word3 0.334 0.241 0.324 0.188
    ...
    word9 0.334 0.241 0.324 0.188

gensim库添加了一个模块，可以用来将glove格式的词向量转为word2vec的词向量，具体操作如下：

In [14]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
# 输入文件
glove_file = datapath('/Users/luoyonggui/Documents/temp/GloVe/vectors.txt')
# 输出文件
tmp_file = get_tmpfile("test_word2vec.txt")

# call glove2word2vec script
# default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>

# 开始转换
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)

# 加载转化后的文件
model = KeyedVectors.load_word2vec_format(tmp_file)

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [16]:
type(model)

gensim.models.keyedvectors.Word2VecKeyedVectors

In [17]:
model['the'], model['the'].shape

(array([ 0.346661, -0.657651, -0.323899,  1.404478, -0.681525,  0.332714,
        -0.493434,  0.950834,  0.102952,  0.386339,  0.347187, -0.912068,
        -0.137993, -1.246887,  0.062835, -0.035913, -0.365426, -0.219618,
        -0.971765,  1.701459,  0.939117,  0.349692, -0.533264,  1.316008,
        -0.708478,  0.394686,  0.751405,  0.498659,  1.257347, -2.340782,
        -0.162739,  0.501467,  1.076938, -0.577817,  0.409253,  0.1018  ,
         0.634708, -0.831036,  0.519494,  0.491227, -0.013857,  0.245493,
        -0.521764, -1.030283, -0.486644, -0.087301,  1.220634,  1.356461,
         1.723411,  1.3197  ], dtype=float32), (50,))

# nb_export

In [5]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 00_template.ipynb.
Converted algo_dl_keras.ipynb.
Converted algo_ml_shallow_tree_catboost.ipynb.
Converted algo_rs_associated_rules.ipynb.
Converted algo_rs_matrix.ipynb.
Converted algo_seq_embeding.ipynb.
Converted algo_seq_tfidf.ipynb.
Converted engineering_nbdev.ipynb.
Converted engineering_panel.ipynb.
Converted index.ipynb.


In [7]:
!nbdev_build_docs

No notebooks were modified
converting /Users/luoyonggui/PycharmProjects/nbdevlib/index.ipynb to README.md
